In [10]:
import pandas as pd
import numpy as np
import math

In [11]:
# definir arquivo Pickle de entrada
#arquivo = 'afastamentos-nomes-sexo-preparado.pkl.bz2'
#arquivo = 'cadastro-nomes-sexo-preparado.pkl.bz2'
arquivo = 'extra/nome_sexo_pf-preparado.pkl.bz2'

In [12]:
df = pd.read_pickle(arquivo)
df.head()

,SEXO,QTDE,L0,L1,L2,L3,L4,L5,L6,L7,L8,L9,L10,L11,L12,L13,L14,L15
NOME,,,,,,,,,,,,,,,,,,
MARIA,0,16078311,1,9,18,1,13,0,0,0,0,0,0,0,0,0,0,0
JOSE,1,8698204,5,19,15,10,0,0,0,0,0,0,0,0,0,0,0,0
JOAO,1,4402784,15,1,15,10,0,0,0,0,0,0,0,0,0,0,0,0
ANTONIO,1,3956710,15,9,14,15,20,14,1,0,0,0,0,0,0,0,0,0
ANA,0,3643117,1,14,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [13]:
%%time
df2 = df.head(50000).copy()
#df2 = df.copy()
menor = df2['QTDE'].min()
maior = df2['QTDE'].max()
print(df2.shape[0], menor, maior, math.ceil(np.sqrt(maior / menor)))
#df2['TAM'] = df2['QTDE'].apply(lambda x: math.ceil(np.sqrt(x / menor)))
df2['TEMP'] = df2[['SEXO', 'QTDE']].apply(
    lambda x: [x[0]] * math.ceil(np.sqrt(x[1] / menor)), axis=1)
dfe = df2.explode('TEMP').drop(['TEMP'], axis=1) #.reset_index()
dfe.head(1000)

# usar os dados expandidos invés dos agregados
dfo = df # guardar o dataframe original
df = dfe # apontar para o dataframe expandido

50000 89 16078311 426
CPU times: user 445 ms, sys: 1.4 ms, total: 446 ms
Wall time: 447 ms


In [14]:
X = df.drop(['SEXO', 'QTDE'], axis=1) # tudo, exceto a coluna alvo
y = df['SEXO'] # apenas a coluna alvo
print('Forma dos dados originais:', X.shape, y.shape)

Forma dos dados originais: (187327, 16) (187327,)


In [34]:
# dividir dados em treino e teste

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)

print("Shapes:", X_train.shape, X_test.shape, y_train.shape, y_test.shape)

Shapes: (125509, 16) (61818, 16) (125509,) (61818,)


https://automl.github.io/auto-sklearn/master/manual.html

In [35]:
# https://automl.github.io/auto-sklearn/master/installation.html

#!curl -k https://raw.githubusercontent.com/automl/auto-sklearn/master/requirements.txt | xargs -n 1 -L 1 pip3 install
#!pip3 install auto-sklearn

In [36]:
import autosklearn.classification

In [37]:
X_train.head()

,L0,L1,L2,L3,L4,L5,L6,L7,L8,L9,L10,L11,L12,L13,L14,L15
NOME,,,,,,,,,,,,,,,,
BIANO,15,14,1,9,2,0,0,0,0,0,0,0,0,0,0,0
MARCIANA,1,14,1,9,3,18,1,13,0,0,0,0,0,0,0,0
SUELEN,14,5,12,5,21,19,0,0,0,0,0,0,0,0,0,0
GUILHERME,5,13,18,5,8,12,9,21,7,0,0,0,0,0,0,0
VALDINHO,15,8,14,9,4,12,1,22,0,0,0,0,0,0,0,0


In [38]:
y_train[:5]

NOME
BIANO        1
MARCIANA     0
SUELEN       0
GUILHERME    1
VALDINHO     1
Name: SEXO, dtype: uint8

In [39]:
automl = autosklearn.classification.AutoSklearnClassifier(
    time_left_for_this_task=20*60,
    per_run_time_limit=2*60,
    n_jobs=-1,
    memory_limit=2048,
    seed=42,
    include_preprocessors=["no_preprocessing"]
)
automl

AutoSklearnClassifier(include_preprocessors=['no_preprocessing'],
                      memory_limit=2048, n_jobs=-1, per_run_time_limit=120,
                      seed=42, time_left_for_this_task=1200)

In [40]:
%%time
automl.fit(X_train, y_train)

CPU times: user 8min 47s, sys: 19.1 s, total: 9min 6s
Wall time: 19min 58s


AutoSklearnClassifier(include_preprocessors=['no_preprocessing'],
                      memory_limit=2048, n_jobs=-1, per_run_time_limit=120,
                      seed=42, time_left_for_this_task=1200)

In [41]:
print(automl.show_models())

[(0.540000, SimpleClassificationPipeline({'balancing:strategy': 'weighting', 'classifier:__choice__': 'adaboost', 'data_preprocessing:categorical_transformer:categorical_encoding:__choice__': 'no_encoding', 'data_preprocessing:categorical_transformer:category_coalescence:__choice__': 'no_coalescense', 'data_preprocessing:numerical_transformer:imputation:strategy': 'mean', 'data_preprocessing:numerical_transformer:rescaling:__choice__': 'robust_scaler', 'feature_preprocessor:__choice__': 'no_preprocessing', 'classifier:adaboost:algorithm': 'SAMME.R', 'classifier:adaboost:learning_rate': 0.7611214587959946, 'classifier:adaboost:max_depth': 4, 'classifier:adaboost:n_estimators': 479, 'data_preprocessing:numerical_transformer:rescaling:robust_scaler:q_max': 0.9953771761327067, 'data_preprocessing:numerical_transformer:rescaling:robust_scaler:q_min': 0.16279981537880428},
dataset_properties={
  'task': 1,
  'sparse': False,
  'multilabel': False,
  'multiclass': False,
  'target_type': 'cla

In [42]:
print(automl.sprint_statistics())

auto-sklearn results:
  Dataset name: 3a647388-c02d-11eb-8332-7759783c0c05
  Metric: accuracy
  Best validation score: 0.934062
  Number of target algorithm runs: 200
  Number of successful target algorithm runs: 98
  Number of crashed target algorithm runs: 30
  Number of target algorithms that exceeded the time limit: 63
  Number of target algorithms that exceeded the memory limit: 9



In [43]:
losses_and_configurations = [
    (run_value.cost, run_key.config_id)
    for run_key, run_value in automl.automl_.runhistory_.data.items()
]
losses_and_configurations.sort()
print("Lowest loss:", losses_and_configurations[0][0])
print("Best configuration:", automl.automl_.runhistory_.ids_config[losses_and_configurations[0][1]])

Lowest loss: 0.06593751509005741
Best configuration: Configuration:
  balancing:strategy, Value: 'none'
  classifier:__choice__, Value: 'adaboost'
  classifier:adaboost:algorithm, Value: 'SAMME.R'
  classifier:adaboost:learning_rate, Value: 0.25648600064981086
  classifier:adaboost:max_depth, Value: 10
  classifier:adaboost:n_estimators, Value: 103
  data_preprocessing:categorical_transformer:categorical_encoding:__choice__, Value: 'one_hot_encoding'
  data_preprocessing:categorical_transformer:category_coalescence:__choice__, Value: 'no_coalescense'
  data_preprocessing:numerical_transformer:imputation:strategy, Value: 'mean'
  data_preprocessing:numerical_transformer:rescaling:__choice__, Value: 'standardize'
  feature_preprocessor:__choice__, Value: 'no_preprocessing'



In [44]:
# treinar o melhor modelo com todos os dados
automl.fit(X, y)

ValueError: {filepath} already exist. Different seeds should be provided for different jobs.

In [ ]:
# carregar dados de teste: definir arquivo Pickle de entrada
#arquivo = 'afastamentos-nomes-sexo-preparado.pkl.bz2'
arquivo = 'cadastro-nomes-sexo-preparado.pkl.bz2'
#arquivo = 'extra/nome_sexo_pf-preparado.pkl.bz2'

In [ ]:
df2 = pd.read_pickle(arquivo)
df2.head()

In [ ]:
df2.shape

In [ ]:
X_test = df2.drop(['SEXO', 'QTDE'], axis=1) # tudo, exceto a coluna alvo
y_test = df2['SEXO'] # apenas a coluna alvo

In [ ]:
y_pred = automl.predict(X_test)
y_pred

In [ ]:
from sklearn.metrics import accuracy_score

print("Accuracy score: %.2f %%" % (accuracy_score(y_test, y_pred) * 100))

In [ ]:
from sklearn.metrics import confusion_matrix

print("Confusion matrix:\n", confusion_matrix(y_test, y_pred))